In [1]:
import os, sys
import pandas as pd
import numpy as np
import datetime as dt
from pprint import pprint
import csv
import itertools as it
from prettytable import PrettyTable
# from nanrms import *
# from handover import *
# from mask import *
import matplotlib.pylab as plt
import seaborn as sns

pdir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # for jupyter-notebook
sys.path.insert(1, pdir)
from myutils import *

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度
# plt.style.use('ggplot')

pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 200)

def set_data(df):
    df['seq'] = df['seq'].astype('Int32')
    # df['rpkg'] = df['rpkg'].astype('Int8')
    df['frame_id'] = df['frame_id'].astype('Int32')
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['xmit_time'] = pd.to_datetime(df['xmit_time'])
    df['arr_time'] = pd.to_datetime(df['arr_time'])
    df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
    df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
    df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
    df['lost'] = df['lost'].astype('boolean')
    df['excl'] = df['excl'].astype('boolean')
    df['latency'] = df['latency'].astype('float32')
    return df

datadir = "/Users/jackbedford/Desktop/MOXA/Code/data"
path = "./temp"
exps = {
    "2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q": ["#{:02d}".format(i+1) for i in range(2)],
}
title = 'All_LTE_B1_B3'
_devices = [
    # ["sm00", "sm01", "sm02", "sm03", "sm04", "sm05", "sm06", "sm07",],
    [f"qc0{i}" for i in range(4)]
]
_schemes = [
    # ["All", "B3", "B7", "B8", "B3B7", "B3B8", "B7B8", "LTE",],
    ["All", "LTE", "B1", "B3",],
]

In [3]:
i = -1
dfs_ul = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ul.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_uplk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_ul[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_ul[i][j] = dfs_ul[i][j][(dfs_ul[i][j]['seq'] >= st) & (dfs_ul[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ul))

i = -1
dfs_dl = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_dl.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df = set_data(df)
            dfs_dl[i].append(df.copy())
            st.append(df['seq'].array[0])
            et.append(df['seq'].array[-1])
        st, et = max(st), min(et)
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            dfs_dl[i][j] = dfs_dl[i][j][(dfs_dl[i][j]['seq'] >= st) & (dfs_dl[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_dl))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm00/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm01/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm02/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm03/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm04/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm05/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm06/#01/data/udp_uplk_loss_latency.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-04/_Bandlock_8_Schemes_Phone/sm07/#01/data/udp_uplk_loss_latency.csv True
/Users/j

In [2]:
i = -1
dfs_ho = []
for (exp, traces), devices, schemes in zip(exps.items(), _devices, _schemes):
# for date, traces in dates.items():
    for trace in traces:
        i += 1
        dfs_ho.append([])
        st, et = [], []
        for j, (dev, schm) in enumerate(zip(devices, schemes)):
            # data = os.path.join(datadir, date, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            dbpath = os.path.join(datadir, exp, dev, trace, 'data')
            data = os.path.join(dbpath, [s for s in os.listdir(dbpath) if s.startswith('diag_log') and s.endswith('_rrc.csv')][0])
            # data = os.path.join(datadir, exp, dev, trace, 'data', 'udp_dnlk_loss_latency.csv')
            print(data, os.path.isfile(data))
            # print(i, j)
            df = pd.read_csv(data)
            df, _, _ = mi_parse_ho(df, tz=8)
            df['ho_type0'] = df['ho_type'].astype('string')
            df.loc[df['cause'].notna(), 'ho_type0'] = df['ho_type'].astype('string') + '_' + df['cause'].astype('string')
            df['ho_type0'] = df['ho_type0'].astype('category')
            dfs_ho[i].append(df.copy())
        #     st.append(df['seq'].array[0])
        #     et.append(df['seq'].array[-1])
        # st, et = max(st), min(et)
        # for j, (dev, schm) in enumerate(zip(devices, schemes)):
        #     dfs_ho[i][j] = dfs_ho[i][j][(dfs_ho[i][j]['seq'] >= st) & (dfs_ho[i][j]['seq'] <= et)].reset_index(drop=True)
print(len(dfs_ho))

/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc00/#01/data/diag_log_qc00_2023-04-17_17-01-08_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc01/#01/data/diag_log_qc01_2023-04-17_17-01-08_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc02/#01/data/diag_log_qc02_2023-04-17_17-01-08_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc03/#01/data/diag_log_qc03_2023-04-17_17-01-08_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc00/#02/data/diag_log_qc00_2023-04-17_17-09-48_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc01/#02/data/diag_log_qc01_2023-04-17_17-09-48_rrc.csv True
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/_Bandlock_Udp_All_LTE_B1_B3_RM500Q/qc02/#02/data/diag

In [3]:
dfs_ho[0][1]

,start,end,ho_type,intr,sPCI,sFreq,tPCI,tFreq,snrPCI,tnrPCI,cause,others,st_scel,ePCI,ECI,eNB,BID,DL_Freq,DL_BW,UL_Freq,UL_BW,ePCI1,ECI1,eNB1,BID1,DL_Freq1,DL_BW1,UL_Freq1,UL_BW1,nrPCI,nrFreq,nrPCI1,nrFreq1,Timestamp,type_id,ho_type0
0,2023-04-17 17:01:10.054661,2023-04-17 17:01:10.194443,Conn_Setup,0.139782,<NA>,<NA>,394,3050,<NA>,<NA>,NaN,<NA>,0,394,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,394,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:01:10.054661,RRC_OTA_Handover_Parsing,Conn_Setup
1,2023-04-17 17:02:02.766732,2023-04-17 17:02:02.791435,LTE_HO,0.024703,394,3050,178,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,394,<NA>,<NA>,<NA>,3050,NaN,<NA>,NaN,178,54562072,213133,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:02.766732,RRC_OTA_Handover_Parsing,LTE_HO
2,2023-04-17 17:02:10.043377,2023-04-17 17:02:10.068182,LTE_HO,0.024805,178,3050,122,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,178,54562072,213133,7,3050,20 MHz,21050,20 MHz,122,54303522,212123,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:10.043377,RRC_OTA_Handover_Parsing,LTE_HO
3,2023-04-17 17:02:13.826439,2023-04-17 17:02:13.850311,LTE_HO,0.023872,122,3050,130,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,122,54303522,212123,7,3050,20 MHz,21050,20 MHz,130,54303512,212123,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:13.826439,RRC_OTA_Handover_Parsing,LTE_HO
4,2023-04-17 17:02:23.720564,2023-04-17 17:02:23.744231,LTE_HO,0.023667,130,3050,19,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,130,54303512,212123,7,3050,20 MHz,21050,20 MHz,19,54547480,213076,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:23.720564,RRC_OTA_Handover_Parsing,LTE_HO
5,2023-04-17 17:02:24.983854,2023-04-17 17:02:25.007092,LTE_HO,0.023238,19,3050,55,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,19,54547480,213076,7,3050,20 MHz,21050,20 MHz,55,54058520,211166,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:24.983854,RRC_OTA_Handover_Parsing,LTE_HO
6,2023-04-17 17:02:25.642043,2023-04-17 17:02:25.671358,LTE_HO,0.029315,55,3050,6,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,55,54058520,211166,7,3050,20 MHz,21050,20 MHz,6,54302990,212121,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:25.642043,RRC_OTA_Handover_Parsing,LTE_HO
7,2023-04-17 17:02:27.077736,2023-04-17 17:02:27.094174,LTE_HO,0.016438,6,3050,130,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,6,54302990,212121,7,3050,20 MHz,21050,20 MHz,130,54303512,212123,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:27.077736,RRC_OTA_Handover_Parsing,LTE_HO
8,2023-04-17 17:02:29.981781,2023-04-17 17:02:30.006378,LTE_HO,0.024597,130,3050,11,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,130,54303512,212123,7,3050,20 MHz,21050,20 MHz,11,54547470,213076,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:29.981781,RRC_OTA_Handover_Parsing,LTE_HO
9,2023-04-17 17:02:31.969579,2023-04-17 17:02:31.986321,LTE_HO,0.016742,11,3050,55,3050,<NA>,<NA>,NaN,Set up 2 SCell.,2,11,54547470,213076,7,3050,20 MHz,21050,20 MHz,55,54058520,211166,7,3050,20 MHz,21050,20 MHz,<NA>,<NA>,<NA>,<NA>,2023-04-17 17:02:31.969579,RRC_OTA_Handover_Parsing,LTE_HO


In [37]:
for i in range(len(dfs_ho)):
    for j in range(len(devices)):
        df = dfs_ho[i][j].copy()
        if sum(df['cause'] == 'rlc-MaxNumRetx (2)') > 0:
            print(f'sm{j}', f'trace#{i+1}', '∆')
        else:
            print(f'sm{j}', f'trace#{i+1}', 'O')

sm0 trace#1 O
sm1 trace#1 ∆
sm2 trace#1 ∆
sm3 trace#1 ∆
sm4 trace#1 ∆
sm5 trace#1 ∆
sm6 trace#1 ∆
sm7 trace#1 O
sm0 trace#2 ∆
sm1 trace#2 ∆
sm2 trace#2 O
sm3 trace#2 O
sm4 trace#2 ∆
sm5 trace#2 ∆
sm6 trace#2 ∆
sm7 trace#2 O
sm0 trace#3 O
sm1 trace#3 ∆
sm2 trace#3 ∆
sm3 trace#3 ∆
sm4 trace#3 ∆
sm5 trace#3 ∆
sm6 trace#3 ∆
sm7 trace#3 O
sm0 trace#4 ∆
sm1 trace#4 O
sm2 trace#4 O
sm3 trace#4 O
sm4 trace#4 O
sm5 trace#4 O
sm6 trace#4 O
sm7 trace#4 O
sm0 trace#5 O
sm1 trace#5 ∆
sm2 trace#5 ∆
sm3 trace#5 ∆
sm4 trace#5 ∆
sm5 trace#5 O
sm6 trace#5 O
sm7 trace#5 O
sm0 trace#6 ∆
sm1 trace#6 ∆
sm2 trace#6 ∆
sm3 trace#6 ∆
sm4 trace#6 ∆
sm5 trace#6 ∆
sm6 trace#6 ∆
sm7 trace#6 O
sm0 trace#7 O
sm1 trace#7 O
sm2 trace#7 O
sm3 trace#7 O
sm4 trace#7 O
sm5 trace#7 O
sm6 trace#7 O
sm7 trace#7 O
